In [1]:
import os

In [2]:
%pwd

'/home/ubuntu/Chicken-Disease-Classification/research'

In [3]:
os.chdir('..')
%pwd

'/home/ubuntu/Chicken-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [5]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories
from ChickenDiseaseClassifier.config.configuration import *

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path= CONFIG_FILE_PATH,
            params_file_path= PARAMS_FILE_PATH
            ):


        print(f"Loading configuration from {config_file_path}")
        print(f"Loading parameters from {params_file_path}")

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])





    # data ingestion configuration 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.data_ingestion_root])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.data_ingestion_root,
            source_url=config.source_url,
            local_data_file =config.local_data_file,
            unzip_dir =config.unzip_dir,
        )
        return data_ingestion_config


    # prepare model configuration 
    def get_prepare_model_config(self) -> PrepareBaseModelConfig:
        config= self.config.prepare_base_model
        create_directories([config.prepare_base_model_root])

        prepare_base_model_config= PrepareBaseModelConfig(
            root_dir= Path(config.prepare_base_model_root),
            base_model_path= Path(config.base_model_path),
            updated_base_model_path= Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            # params_batch_size=self.params.BATCH_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            # params_epochs=self.params.EPOCHS,
            params_classes=self.params.CLASSES,
            params_weights=self.params.WEIGHTS,
            params_learning_rate=self.params.LEARNING_RATE, 

        )
        return prepare_base_model_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2023-08-24 13:28:57.737748: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 13:28:57.784459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-08-24 13:28:58.412575: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model= tf.keras.applications.vgg16.VGG16(
            input_shape= self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top
        )
        self.save_model(path= self.config.base_model_path, model= self.model)



    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif(freeze_till is not None) and (freeze_till > 0):
            for lambda_layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in= tf.keras.layers.Flatten()(model.output)
        prediction= tf.keras.layers.Dense(
            units= classes,
            activation= "softmax"
        )(flatten_in)

        full_model= tf.keras.models.Model(inputs= model.input, outputs= prediction)
        full_model.compile(
            optimizer= tf.keras.optimizers.Adam(learning_rate= learning_rate),
            loss= tf.keras.losses.CategoricalCrossentropy(),
            metrics= ["accuracy"]
        )

        full_model.summary()
        return full_model


    def update_base_model(self):
        self.full_model= self.prepare_full_model(
            model= self.model,
            classes= self.config.params_classes,
            freeze_all= True,
            freeze_till= None,
            learning_rate= self.config.params_learning_rate
        )

        self.save_model(path= self.config.updated_base_model_path, model= self.full_model)
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model) -> None:
        model.save(path)



In [9]:
try:
    config= ConfigurationManager()
    prepare_base_model_config= config.get_prepare_model_config()
    prepare_base_model= PrepareBaseModel(
        config= prepare_base_model_config
    )
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise(e)


Loading configuration from config/config.yaml
Loading parameters from params.yaml
[2023-08-24 13:28:59,279: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-24 13:28:59,281: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-24 13:28:59,281: INFO: common: created directory at: artifacts]
[2023-08-24 13:28:59,282: INFO: common: created directory at: artifacts/prepare_base_model]


2023-08-24 13:29:04.891610: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13719 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:18:00.0, compute capability: 7.5
2023-08-24 13:29:04.894087: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13719 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:19:00.0, compute capability: 7.5
2023-08-24 13:29:04.896270: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13719 MB memory:  -> device: 2, name: Tesla T4, pci bus id: 0000:35:00.0, compute capability: 7.5
2023-08-24 13:29:04.898428: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13719 MB memory:  -> device: 3, name: Tesla T4, pci bus id: 0000:36:00.0, compute capability: 7.5
2023-08-

[2023-08-24 13:29:05,483: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                          

/home/ubuntu/Chicken-Disease-Classification/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
